In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import iqr
from statistics import mean

In [3]:
os.getcwd()
os.chdir('C:\\Users\\STSC\Desktop\\Fall 2020 Syllabus\\Ban612')
house_sample= pd.read_csv('data2.csv')

In [4]:
#Remove spaces in colmn names,rename index, fill in Not rated as O and remove string car from column garage.
house_sample.columns = house_sample.columns.str.strip()
house_sample.rename_axis(index='Seq', inplace=True)
house_sample = house_sample.replace(['Not rated'],0)
house_sample['garage'] = house_sample['garage'].str.replace(r'\D', '')
house_sample['garage'].fillna(0, inplace=True)

In [5]:
#Check to see if column 0-4 have any null values and then count how many. Drop those columns. Then fill in any other missning null with O
x = house_sample.index[house_sample.iloc[:,:4].isnull().any(axis=1)]
rows_missing_val = len(x)
house_sample.drop(house_sample.index[house_sample.iloc[:,:4].isnull().any(axis=1)], inplace=True)
y = house_sample.index[house_sample.isnull().any(axis=1)]
house_sample = house_sample.fillna(0)

In [6]:
#Shows count of how many rows had a null value between columns 0-4
if rows_missing_val >0:
    print('Enter {} new housing url to make sure we have 500 data points that is missing no info from columns 1-7.'.format(rows_missing_val))

Enter 7 new housing url to make sure we have 500 data points that is missing no info from columns 1-7.


In [7]:
#Remove data where year of home was built is zero
house_sample.drop(house_sample.index[house_sample['built']==0], inplace=True)

In [8]:
#Remove $,commas,spacing from total column and change it to a float. Also remove left and right side spaces from city column.
house_sample['total'] = house_sample['total'].str.replace(',', '').str.replace('$', '').str.replace(' ','').astype(float)
house_sample['city']= house_sample['city'].str.strip()
house_sample.columns = house_sample.columns.str.replace(' ','_')

In [9]:
#Replace invalid data
house_sample.isin(['10-Sep']).any()

house_sample['elementary_school'] = house_sample['elementary_school'].replace(['10-Sep'],8)

In [10]:
#Change dtypes and view them
house_sample = house_sample.astype({'total': 'int64', 'garage': 'int32'})
house_sample.iloc[:,[13,14,15]] = house_sample.iloc[:,[13,14,15]].astype(np.float64)
house_sample.dtypes

total                  int64
bed                    int64
bath                 float64
square                 int64
lot                  float64
price                 object
type                  object
built                float64
garage                 int32
address               object
city                  object
state                 object
zip                    int64
elementary_school    float64
middle_school        float64
high_school          float64
Collateral            object
CoreLogic             object
Quantarium            object
dtype: object

In [11]:
#Summary statistics of numeric columns
house_stats =house_sample.describe().round(1)
house_stats

,total,bed,bath,square,lot,built,garage,zip,elementary_school,middle_school,high_school
count,493.0,493.0,493.0,493.0,493.0,493.0,493.0,493.0,493.0,493.0,493.0
mean,1288995.7,3.0,2.2,1696.0,6222.9,1960.8,1.4,94526.2,4.8,5.2,5.2
std,837885.0,1.1,0.8,758.7,16444.1,31.6,1.0,301.3,3.0,2.7,2.6
min,165000.0,1.0,1.0,410.0,0.0,1881.0,0.0,94102.0,0.0,0.0,0.0
25%,799000.0,2.0,1.5,1184.0,1245.0,1941.0,1.0,94401.0,2.0,4.0,4.0
50%,1149000.0,3.0,2.0,1525.0,4370.0,1959.0,2.0,94544.0,5.0,6.0,5.0
75%,1588000.0,4.0,2.5,1930.0,6310.0,1985.0,2.0,94608.0,7.0,7.0,7.0
max,8495000.0,8.0,6.0,5084.0,267022.0,2019.0,13.0,95054.0,10.0,9.0,9.0


In [12]:
house_sample.columns = house_sample.columns.str.replace(' ','_')
house_sample['price']=house_sample['price'].str.replace(',', '')

In [13]:
house_sample.head()

,total,bed,bath,square,lot,price,type,built,garage,address,city,state,zip,elementary_school,middle_school,high_school,Collateral,CoreLogic,Quantarium
Seq,,,,,,,,,,,,,,,,,,,
1,1485000,2,2.0,1100,22651.0,1350,Condo,2014.0,1,300 Ivy St Apt 312,San Francisco,CA,94102,0.0,2.0,6.0,$1443000 $1229343 $1277894,0,$1393492 $1217410 $1213049
3,1399000,2,2.5,1930,0.0,725,Condo,2000.0,1,725 Tehama St Unit 3,San Francisco,CA,94103,0.0,2.0,5.0,$1491000 $1142108 $1327877,0,$1415477 $1249816 $1273104
4,1195000,2,2.5,1537,0.0,777,Condo,1999.0,2,786 Minna St Apt 8,San Francisco,CA,94103,0.0,2.0,5.0,$1258000 $1143008 $1328923,0,$1220363 $1240076 $1242719
5,1095000,2,2.0,1161,0.0,943,Condo,2001.0,1,590 6th St Apt 211,San Francisco,CA,94103,0.0,2.0,5.0,$1134000 $1038608 $1207542,0,$1143357 $1165112 $1176763
6,769000,2,2.0,796,0.0,966,Condo/Townhome/Row Home/Co-Op,2013.0,1,537 Natoma St Unit 204,San Francisco,CA,94103,0.0,2.0,5.0,$1046000 $946807 $1100809,0,$951350 $1018313 $947081


In [14]:
house_sample['price'] = house_sample['price'].fillna(house_sample.total/house_sample.square)

In [15]:
#zip_dum = house_sample.get_dummies()
city_dum = pd.get_dummies(house_sample.city)
house_sample3 = pd.concat([house_sample.drop(columns=['city','address','Collateral','CoreLogic','Quantarium','state','zip']),city_dum], axis=1)

In [16]:
#Remove cities Berkeley,Alameda and row with 4 different home type listed.
house_sample3 = house_sample3[(house_sample3['type'] !='Condo/Townhome/Row Home/Co-Op')]
house_sample3[house_sample3['type']=='Condo/Townhome/Row Home/Co-Op']

,total,bed,bath,square,lot,price,type,built,garage,elementary_school,middle_school,high_school,Alameda,Berkeley,Hayward,Oakland,Pleasanton,San Francisco,San Mateo,Santa Clara
Seq,,,,,,,,,,,,,,,,,,,,


In [17]:
#Create a list of unique type of homes and cities
unq_homes = house_sample3['type'].unique()
home_category = dict(zip(unq_homes, (1,4,3,2,0)))
unq_homes

array(['Condo', 'Multi-Family Home', 'Single Family Home', 'Townhome',
       'Mfd/Mobile Home'], dtype=object)

In [18]:
#Remove data where year built is 0, copy df, remove left and right spaces from city.Turn type of home into categorical variable
house_sample3['built']= house_sample3['built'].fillna(0)
house_w_category = house_sample3.copy()
house_w_category['type']= house_w_category['type'].apply(lambda x: home_category[x])

In [19]:
#View type as categorical
house_w_category['type'].unique()

array([1, 4, 3, 2, 0], dtype=int64)

In [20]:
#View df correlation oftotal to each numerical column
house_w_category.corrwith(house_w_category['total'])

total                1.000000
bed                  0.353410
bath                 0.469739
square               0.606037
lot                  0.017002
type                 0.182624
built               -0.119114
garage               0.102903
elementary_school    0.220249
middle_school        0.266193
high_school          0.264091
Alameda             -0.033797
Berkeley             0.000180
Hayward             -0.289119
Oakland             -0.174966
Pleasanton          -0.041779
San Francisco        0.275358
San Mateo            0.205487
Santa Clara         -0.028637
dtype: float64

In [21]:
import torch
import torch.optim as optim
import torch.nn as nn
import sklearn
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F



In [22]:
house_w_category = sklearn.utils.shuffle(house_w_category)
house_w_category


,total,bed,bath,square,lot,price,type,built,garage,elementary_school,middle_school,high_school,Alameda,Berkeley,Hayward,Oakland,Pleasanton,San Francisco,San Mateo,Santa Clara
Seq,,,,,,,,,,,,,,,,,,,,
258,325000,3,2.0,1602,0.0,203,0,2005.0,0,0.0,5.0,4.0,0,0,1,0,0,0,0,0
48,5400000,4,3.5,3600,2848.0,1500,3,1900.0,0,9.0,7.0,2.0,0,0,0,0,0,1,0,0
247,799000,4,2.0,1879,5035.0,425,3,1952.0,1,2.0,0.0,5.0,0,0,1,0,0,0,0,0
7,1195000,3,1.0,1320,0.0,905,1,1907.0,1,0.0,0.0,0.0,0,0,0,0,0,1,0,0
424,1249950,3,2.5,1342,5848.0,931,3,1946.0,2,6.0,5.0,6.0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,718000,3,1.0,1324,7700.0,618,3,1905.0,0,6.0,6.0,5.0,0,0,0,0,0,0,0,1
365,799000,3,3.0,2352,8639.0,340,3,1983.0,2,0.0,0.0,0.0,0,0,0,1,0,0,0,0
377,1050000,3,2.0,1396,5400.0,752,3,1913.0,2,2.0,3.0,5.0,0,0,0,1,0,0,0,0


In [23]:
 ###########homework 10 starts######################################
    
X_train, X_test, y_train, y_test = train_test_split(house_w_category.iloc[:,1:],house_w_category.iloc[:,0], test_size=0.2, random_state=1)

In [24]:
y_train = y_train.to_numpy()
y_test= y_test.to_numpy()
y_train= y_train.reshape(392,1)
y_test=y_test.reshape(99,1)

In [25]:
len(y_test)

99

In [26]:
#Scale
scaler = StandardScaler()
# transform data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train =scaler.fit_transform(y_train)
y_test= scaler.transform(y_test)
print(type(X_train))

<class 'numpy.ndarray'>


In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).long().to(device)
y_test = torch.from_numpy(y_test).long().to(device)

In [28]:
len(y_train)

392

In [29]:
class Two_Layer_NN(nn.Module):
    def __init__(self):
        super().__init__()
        # Full connected nn
        self.fc1 = nn.Linear(19, 10)       
        self.fc2 = nn.Linear(10, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x= self.fc2(x)
    
        return x

# Initialize it
Two_Layer_Model = Two_Layer_NN()
Two_Layer_Model

Two_Layer_NN(
  (fc1): Linear(in_features=19, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)

In [30]:
loss_fn = F.mse_loss
def d_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    return train_step

In [31]:
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

In [32]:
optimizer = torch.optim.Adam(Two_Layer_Model.parameters(), lr=0.0001)
train_step = d_train_step(Two_Layer_Model, loss_fn, optimizer)


for epoch in range(1000):
    losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # recall that train_step returns a scalar for loss
        loss = train_step(x_batch, y_batch)
        
        
        losses.append(loss)
    if (epoch+1) % 100 == 0:
        print(sum(losses)/16)

0.5150840894784778
0.34241548413410783
0.23999606852885336
0.17925392626784742
0.13890586397610605
0.11202195653459057
0.08868993551004678
0.07374115771381184
0.06404247845057398
0.05706773657584563


In [33]:
train_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

optimizer = torch.optim.Adam(Two_Layer_Model.parameters(), lr=0.0001)
train_step = d_train_step(Two_Layer_Model, loss_fn, optimizer)


for epoch in range(1000):
    losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # recall that train_step returns a scalar for loss
        loss = train_step(x_batch, y_batch)
        
        
        losses.append(loss)
    if (epoch+1) % 100 == 0:
        print(sum(losses)/16)

0.021521395654417574
0.017073770286515355
0.013326142623554915
0.010199987096711993
0.007942858508613426
0.007763674831949174
0.00671049885568209
0.005362955249438528
0.005549959751078859
0.004823958937777206
